# 4. Logistic Regression

## Loading Image datasetset

In [ ]:
import numpy as np

# Load the .npz file
data = np.load('dataset_features.npz')

# List all arrays within the .npz file
print(data.files)

# Access individual arrays by their names
X_train = data['trainset_features']
y_train = data['trainset_labels']

X_val = data['validset_features']
y_val = data['validset_labels']

X_test = data['testset_features']
y_test = data['testset_labels']

class_labels = data['class_labels']

## Logistic regression model training

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from util import decode_class

# Create a k-NN pipeline
logreg_pipe = Pipeline(
    [("scaler", StandardScaler()), 
     ("logreg", LogisticRegression(multi_class='multinomial', solver='saga', penalty='none'))]
)

# Fit it to train data
logreg_pipe.fit(X_train, decode_class(y_train))

## Model evaluation

Accuracy score on train, validation and test sets

In [ ]:
print('Model Accuracy:')
acc_train = logreg_pipe.score(X_train, decode_class(y_train))
print(f'On train set: {acc_train:.3f}')
acc_val = logreg_pipe.score(X_val, decode_class(y_val))
print(f'On valid set: {acc_val:.3f}')
acc_test = logreg_pipe.score(X_test, decode_class(y_test))
print(f'On test  set: {acc_test:.3f}')

Classification report

In [ ]:
from sklearn.metrics import classification_report

# Classification report
y_test_preds = logreg_pipe.predict(X_test)

print(classification_report(y_true=decode_class(y_test), y_pred=y_test_preds, target_names=class_labels))

## Model coefficients visualization

In [ ]:
coefficients = logreg_pipe.named_steps['logreg'].coef_ 
coefficients.shape

Visualizations of the coefficients as a heatmap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(np.abs(coefficients), annot=False, cbar=True)
ax.set_yticklabels(class_labels, rotation=0)
ax.set_xlabel("Feature Index")
plt.title("Logistic Regression Coefficients Heatmap")

Next, we look for the indices of the top 5 largest coefficients (absolute values) for each class

In [ ]:
import pandas as pd

top_features = {}
for class_index, class_coefficients in enumerate(coefficients):
    
    # Get the indices of the top 5 largest coefficients for the current class
    largest_indices = np.argsort(-np.abs(class_coefficients))[:5]
    top_features[f"{class_labels[class_index]}"] = largest_indices


top_features_df = pd.DataFrame.from_dict(top_features, orient='index', columns=[f"Feature {i+1}" for i in range(5)])
print("Top 5 Largest Coefficients for Each Class (Feature Indices):")
print(top_features_df)

__Observation:__

- The coefficients obtained in Task 1 were:

    bike:        **[148, 801, 183, 1094, 54]**  
    car:         **[1098, 291, 183, 660, 257]**  
    motorcycle:  **[1043, 505, 898, 1122, 1120]**  
    other:       **[580, 529, 734, 279, 411]**  
    truck:       **[1051, 335, 580, 714, 1022]**  
    van:         **[893, 466, 1113, 1104, 1022]**  

- We observe that except for a few exceptions most high-level features are different. The shared coefficients are:
    - Bike: **801**  
    - Car: **291**  
    - Motorcycle: **1122**  
    - Other: **None**  
    - Truck: **None**
    - Van: **None**

- This is not really consistent with the exploration in Task 1, although the categories for which there's a common feature are the ones with more samples and slightly better classification scores

## Model regularization

Next we'll apply l2-regularization to the model and fine-tune the parameter (C = 1/lambda) that is the inverse of the strengh of the regularization term in the cost funtion. 

We first start my merging our train and validation sets to then apply cross-validation technique during the parameter grid-search.

In [ ]:
X_crossval = np.concatenate((X_train, X_val), axis=0)
y_crossval = decode_class(np.concatenate((y_train, y_val), axis=0))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define logistic regression model with L2 regularization
log_reg = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, multi_class='multinomial')

# Define grid of regularization strengths to test
param_grid = {
    'C': np.logspace(-6, 3, 20)  # Test values from 10^-4 to 10^4
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring='accuracy',  # Metric for evaluating models
    cv=5,                # 5-fold cross-validation
    n_jobs=-1,           # Use all available processors
    return_train_score=True
)

# Perform grid search
grid_search.fit(X_crossval, y_crossval)

In [ ]:
# Extract results from the GridSearchCV object
results = pd.DataFrame(grid_search.cv_results_)

# Select relevant columns for interpretation
results_df = results[
    [
        'param_C',  
        'mean_train_score',
        'std_train_score', 
        'mean_test_score', 
        'std_test_score'  
    ]
]

# Sort by the validation score for better interpretability
results_df = results_df.sort_values(by='mean_test_score', ascending=False)
results_df

- **mean_train_score** and **std_train_score**: refer to average and standard deviation of the model accuracy obtained over the 5-fold (5x 4/5 of total data) of the train sub-sets derived from the merged train+val dataset.
- **mean_test_score** and **std_test_score**: the same as before but this it represents the score in the 5-fold (5x 1/5 of total data) validation sets are created.
    
- Low standard deviations (**std_train_score** and **std_test_score**) indicate consistent performance.

- High mean validation scores with low standard deviations are desirable for a robust,  well-generalizing model.

- These metrics help diagnose overfitting, underfitting, or data-related issues during hyperparameter tuning.

In [ ]:
# Assuming you have metrics recorded during training, such as accuracy or loss
# Example data (replace with actual metrics from your training process)

results_df = results_df.sort_values(by='param_C')

param_c = results_df['param_C'].tolist()
train_scores_mean = results_df['mean_train_score'].to_numpy()
val_scores_mean = results_df['mean_test_score'].to_numpy()
train_scores_std = results_df['std_train_score'].to_numpy()
val_scores_std = results_df['std_test_score'].to_numpy()
# Replace with validation accuracies

# Plot the curves
plt.figure(figsize=(10, 6))
plt.plot(param_c, train_scores_mean, label="Training Accuracy", marker="o")
plt.plot(param_c, val_scores_mean, label="Validation Accuracy", marker="o")
plt.fill_between(param_c, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2)
plt.fill_between(param_c, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.2)
plt.title("Training and Validation Accuracy")
plt.xlabel("C Parameter")
plt.ylabel("Accuracy")
plt.xscale('log')
plt.legend()
plt.grid()

We observe that for values of C > 0.001 the model starts to overfitting and not gaining anymore capability to generalize. This is indicated by a gap showing in the model accuracy of the training and validation sets.

In [ ]:
#logreg_pipe_tuned = grid_search.best_estimator_
logreg_pipe_tuned = logreg_pipe.set_params(logreg__C=0.005)

# Fit it to train data
logreg_pipe_tuned.fit(X_train, decode_class(y_train))

# Check accuracy on training set
acc_train = logreg_pipe_tuned.score(X_train, decode_class(y_train))

## Evaluation of model with tuned l2-regularization

In [ ]:
print('Model Accuracy:')
acc_train = logreg_pipe_tuned.score(X_train, decode_class(y_train))
print(f'On train set: {acc_train:.3f}')
acc_val = logreg_pipe_tuned.score(X_val, decode_class(y_val))
print(f'On valid set: {acc_val:.3f}')
acc_test = logreg_pipe_tuned.score(X_test, decode_class(y_test))
print(f'On test  set: {acc_test:.3f}')

__Observation:__

- The model accuracy on the test set with the optimized l2-regularization is exactly the same as what was obtained without regularization: accuracy = 0.96
- As can be seen from the training and validation curves, the regularization term doesn't affect the already very high model performance, except when the l2-term becomes excidengly high for C<0.001

Finally we save the accuracy results on the test data set in the pickle file

In [ ]:
import pandas as pd
import pickle


with open('model_accuracy.pickle', 'rb') as file:
    results_acc = pickle.load(file)
    
with open('model_accuracy.pickle', 'wb') as file:
    results_acc.loc[len(results_acc)] = {'model': 'logistic', 'test_accuracy': acc_test}
    pickle.dump(results_acc, file)
    
print(results_acc)
